In [50]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [51]:
from pyspark.sql import SparkSession

# Crie uma instância do SparkSession
spark = SparkSession.builder.appName("atividadePUC").getOrCreate()


In [52]:
# Leitura de um arquivo CSV em um DataFrame
itens_provas = spark.read.csv("enem2020/DADOS/ITENS_PROVA_2020.csv", header=True, sep=";")
microdados_enem = spark.read.csv("enem2020/DADOS/MICRODADOS_ENEM_2020.csv", header=True, sep=";")
dic_micro_dados = spark.read.csv("enem2020/DICIONÁRIO/Dicionário_Microdados_Enem_2020.xlsx", header=True, sep=";")


In [53]:
itens_provas.createOrReplaceTempView('ITENS_PROVA_2020')
microdados_enem.createOrReplaceTempView('MICRODADOS_ENEM_2020')

Quantas mulheres que moram sozinhas, estudaram numa escola no estado de SP fizeram a prova de Ciências Humanas do ENEM em 2020?

#Q005 - 1 = Mora sozinho(a)
#TP_SEXO = 'F'
#SG_UF_ESC = 'sp'
#CO_PROVA_CH <> '' ou Null

In [54]:
pergunta_1 = spark.sql("""
SELECT 
    TP_SEXO as SEXO,
    SG_UF_ESC as UF,
    CO_PROVA_CH,
    CO_UF_PROVA,
    Q005 AS MORA_SOZINHA
FROM 
    MICRODADOS_ENEM_2020
WHERE 
    TP_SEXO = 'F'
    AND SG_UF_ESC = 'SP'
    AND TP_PRESENCA_CH = 1
    AND Q005 = 1;
""")
                       

# Conta o número total de linhas no DataFrame
total_de_linhas = pergunta_1.count()

# Exibe o total de linhas
print("Total de linhas:", total_de_linhas)

Total de linhas: 65


Qual é o município (município da escola) que obteve a maior nota MÉDIA em matemática?

#Group by NO_MUNICIPIO_ESC = nome do municipio
# NU_NOTA_MT = nota matematica


In [55]:
spark.sql("""
SELECT 
    NO_MUNICIPIO_ESC AS MUNICIPIO,
    AVG(NU_NOTA_MT) AS NOTA_MEDIA_MAT,
    COUNT(NU_INSCRICAO) AS CONTAGEM_PARTICPANTES
FROM 
    MICRODADOS_ENEM_2020
WHERE 
    NU_NOTA_MT IS NOT NULL
GROUP BY 
    NO_MUNICIPIO_ESC
ORDER BY 
    NOTA_MEDIA_MAT DESC
""").show()


+--------------------+-----------------+---------------------+
|           MUNICIPIO|   NOTA_MEDIA_MAT|CONTAGEM_PARTICPANTES|
+--------------------+-----------------+---------------------+
|          Novo Xingu|            836.1|                    1|
|      Novo Itacolomi|            823.2|                    1|
|    Coqueiros do Sul|            787.4|                    1|
|           Biquinhas|            784.2|                    1|
| Santa Clara d'Oeste|            738.0|                    1|
| Santana do Manhua�u|            720.2|                    1|
|            Jesu�nia|            714.7|                    1|
|              Vanini|            713.4|                    1|
|        Dolcin�polis|            711.4|                    1|
|      Pedro Teixeira|            703.4|                    1|
|              Flora�|            702.7|                    2|
|           Ubiretama|696.6500000000001|                    2|
|            Populina|            695.0|               

In [56]:
total_participantes = spark.sql("""
SELECT 
   * FROM MICRODADOS_ENEM_2020
""").show()

# Exibe o total de linhas
print("Total de linhas:", total_participantes)


+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

Quantas pessoas que estudaram numa escola em Recife fizeram a prova do ENEM nessa mesma cidade?

In [57]:
total_participantes = spark.sql("""
SELECT 
   * FROM MICRODADOS_ENEM_2020
   where NO_MUNICIPIO_ESC = 'Recife'
""").count()

# Exibe o total de linhas
print("Total de linhas:", total_participantes)


Total de linhas: 10735


Qual é a cidade (município da escola) que possui a MAIOR nota em Ciências Naturais?

In [87]:
spark.sql("""
SELECT 
    NO_MUNICIPIO_ESC AS MUNICIPIO,
    AVG(NU_NOTA_CN) AS NOTA_MEDIA_CN,
    COUNT(NU_INSCRICAO) AS CONTAGEM_PARTICIPANTES
FROM 
    MICRODADOS_ENEM_2020
WHERE 
    NU_NOTA_CN IS NOT NULL
GROUP BY 
    NO_MUNICIPIO_ESC
ORDER BY 
    NOTA_MEDIA_CN DESC;
""").show(50)

+--------------------+-----------------+----------------------+
|           MUNICIPIO|    NOTA_MEDIA_CN|CONTAGEM_PARTICIPANTES|
+--------------------+-----------------+----------------------+
|           Biquinhas|            670.6|                     1|
|    Coqueiros do Sul|            649.7|                     1|
|          Novo Xingu|            647.2|                     1|
|         Passa Vinte|            621.4|                     2|
|Santo Ant�nio do ...|            614.3|                     1|
|           Gon�alves|            608.6|                     1|
|     Porto Vera Cruz|           608.55|                     2|
|Vargem Grande do ...|605.7666666666668|                     3|
|       Monte Formoso|            604.8|                     1|
|      Pedro Teixeira|            595.1|                     1|
|             Aguanil|           594.15|                     2|
|            Montauri|            594.1|                     1|
|        Mariano Moro|            593.3|

In [71]:
spark.sql("""
SELECT NO_MUNICIPIO_ESC, NU_NOTA_CN
FROM MICRODADOS_ENEM_2020
ORDER BY NU_NOTA_CN DESC;
""").show()

+----------------+----------+
|NO_MUNICIPIO_ESC|NU_NOTA_CN|
+----------------+----------+
|     Jo�o Pessoa|     854.8|
|            null|     854.8|
|            null|     854.8|
|        Bras�lia|     854.8|
|            null|     851.1|
|            null|     843.3|
|            null|     843.3|
|            null|     838.7|
|            null|     835.3|
|            null|     834.1|
|            null|     833.5|
|            null|     831.5|
|           Bel�m|     831.5|
|            null|     831.5|
|         Goi�nia|     831.2|
|            null|     829.6|
|            null|     828.9|
|            null|     827.9|
|            null|     827.2|
|            null|     827.2|
+----------------+----------+
only showing top 20 rows



Qual é a nota média em matemática dos alunos cuja mãe possui PELO MENOS o ensino superior completo, do sexo feminino que estudaram numa escola em Belo Horizonte?

In [ ]:
spark.sql("""
SELECT 
    Q002,
    AVG(NU_NOTA_MT) AS NOTA_MEDIA_MAT,
    COUNT(NU_INSCRICAO) AS CONTAGEM_PARTICIPANTES
FROM 
    MICRODADOS_ENEM_2020
WHERE 
    NU_NOTA_MT IS NOT NULL
    AND Q002 IN ('F', 'G')
GROUP BY 
    Q002
ORDER BY 
    NOTA_MEDIA_MAT DESC;
""").show(50)

+----+-----------------+----------------------+
|Q002|   NOTA_MEDIA_MAT|CONTAGEM_PARTICIPANTES|
+----+-----------------+----------------------+
|   G| 594.751494302845|                276015|
|   F|580.2815139587111|                313496|
+----+-----------------+----------------------+



In [ ]:
spark.sql("""
SELECT 
    CASE 
        WHEN Q002 IN ('F', 'G') THEN 'F ou G'
        ELSE Q002
    END AS Q002_UNIFICADO,
    ROUND(AVG(NU_NOTA_MT), 2) AS NOTA_MEDIA_MAT,
    COUNT(NU_INSCRICAO) AS CONTAGEM_PARTICIPANTES
FROM 
    MICRODADOS_ENEM_2020
WHERE 
    NU_NOTA_MT IS NOT NULL
    AND Q002 IN ('F', 'G')
GROUP BY 
    Q002_UNIFICADO
ORDER BY 
    NOTA_MEDIA_MAT DESC;          
""").show()

+--------------+--------------+----------------------+
|Q002_UNIFICADO|NOTA_MEDIA_MAT|CONTAGEM_PARTICIPANTES|
+--------------+--------------+----------------------+
|        F ou G|        587.06|                589511|
+--------------+--------------+----------------------+



Sobre o ENEM 2020, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?

In [74]:
spark.sql("""
SELECT 
    TP_SEXO AS SEXO,
    AVG(NU_NOTA_MT) AS NOTA_MEDIA_MAT,
    COUNT(NU_INSCRICAO) AS CONTAGEM_PARTICPANTES
FROM 
    MICRODADOS_ENEM_2020
WHERE 
    NU_NOTA_MT IS NOT NULL
GROUP BY 
    TP_SEXO
ORDER BY 
    NOTA_MEDIA_MAT DESC
""").show()

+----+-----------------+---------------------+
|SEXO|   NOTA_MEDIA_MAT|CONTAGEM_PARTICPANTES|
+----+-----------------+---------------------+
|   M|550.9087170619391|              1028787|
|   F|500.6864256785913|              1568653|
+----+-----------------+---------------------+



Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola no estado de Santa Catarina e possuem PELO MENOS 1 banheiro em casa?

In [83]:
spark.sql("""
SELECT 
    AVG(NU_NOTA_CH) AS NOTA_MEDIA_CH,
    COUNT(NU_INSCRICAO) AS CONTAGEM_PARTICIPANTES
FROM 
    MICRODADOS_ENEM_2020
WHERE 
    SG_UF_PROVA = 'SC'
    AND Q008 <> 'A'
GROUP BY 
    SG_UF_PROVA
ORDER BY 
    NOTA_MEDIA_CH DESC;
""").show()

+-----------------+----------------------+
|    NOTA_MEDIA_CH|CONTAGEM_PARTICIPANTES|
+-----------------+----------------------+
|534.9679601006472|                117954|
+-----------------+----------------------+



Qual é a média da nota em Ciências Humanas para os alunos do sexo masculino que estudaram numa escola no estado de São Paulo?

In [89]:
spark.sql("""
SELECT 
    AVG(NU_NOTA_CH) AS NOTA_MEDIA_CH,
    COUNT(NU_INSCRICAO) AS CONTAGEM_PARTICIPANTES
FROM 
    MICRODADOS_ENEM_2020
WHERE 
    NU_NOTA_CH IS NOT NULL
    AND TP_SEXO = 'M'
    AND SG_UF_ESC = 'SP'
GROUP BY 
    SG_UF_ESC
ORDER BY 
    NOTA_MEDIA_CH DESC;
""").show(50)

+----------------+----------------------+
|   NOTA_MEDIA_CH|CONTAGEM_PARTICIPANTES|
+----------------+----------------------+
|553.678013870444|                 47439|
+----------------+----------------------+



Quantas pessoas que estudaram numa escola no estado do Rio de Janeiro do sexo masculino obtiveram nota em matemática maior do que 600?

In [91]:
total_participantes = spark.sql("""
SELECT COUNT(NU_INSCRICAO) AS CONTAGEM_PARTICIPANTES
FROM MICRODADOS_ENEM_2020
WHERE SG_UF_ESC = 'RJ'
  AND TP_SEXO = 'M'
  AND NU_NOTA_MT > 600;
""").show()

+----------------------+
|CONTAGEM_PARTICIPANTES|
+----------------------+
|                  8581|
+----------------------+

